# SQLMagic (%sql magic) with Python 

This notebook highlights how you can create a connection to a SQL Server instance with trusted connection, that interacts with Python.

A few notes:
* We are using [ipython-sql](https://github.com/catherinedevlin/ipython-sql) module. Please make sure that you install this in your environment first before you can start using the %sql magic. 
* Get Azure Data Studio here: http://aka.ms/getAzureDataStudio



## 1. Load %sql Magic extension



In [1]:
 %reload_ext sql
 %config SqlMagic.displaycon = False # change this to True to show the connection string for the %sql commands executed against


## 2. Setup connection string

Provide your username and database name

In [ ]:
serverName = input("Enter the SQL Server instance you'd like to connect to: ")
database = input("Enter the database to connect to: ")

import urllib
params = urllib.parse.quote_plus("DRIVER={SQL SERVER};SERVER=" + serverName + ";DATABASE=" + database + ";TRUSTED_CONNECTION=YES")
conn = "mssql+pyodbc:///?odbc_connect=%s" % params
conn

## 3. Connect to SQL Server instance and database

In [3]:
%sql {conn}

## 4. Query sys.databases and filter a database with a name = "master"

This is a simple exapmle of passing a variable from Python to a SQL query.

In [14]:
database_name = "master"
%sql select :database_name
%sql select top 4 name, compatibility_level from sys.databases where name = :database_name

Done.
Done.


name,compatibility_level
master,150


## 5. Create a stored procedure dbo.usp_AddDayToADate

This will create a stored procedure that will add a number of days to a specified date. The default is to to return tomorrow's date. 

In [7]:
%%sql CREATE PROCEDURE dbo.usp_AddDayToADate
(
    @DayNumber INT = 1,
    @Date DATE = NULL
) AS
BEGIN
    SELECT DATEADD(DAY, @DayNumber, ISNULL(@Date, GETDATE())) AS NewDate;
END;

Done.


[]

## 6. Run the stored procedure dbo.usp_AddDayToADate with default input values

In [8]:
%sql EXEC dbo.usp_AddDayToADate

Done.


NewDate
2020-10-19


## 7. Run dbo.usp_AddDayToADate with input parameters from Python

This will add 5 days to the Jan 1, 2020. 

In [9]:
numberOfDays = 5
someDate = '20200101'

%sql EXEC dbo.usp_AddDayToADate :numberOfDays, :someDate

Done.


NewDate
2020-01-06


Use `numberOfDays` set in previous cell as a parameter in another stored proc call.


In [10]:
numberOfDays = 5

%sql EXEC dbo.usp_AddDayToADate :numberOfDays, '20201010'

Done.


NewDate
2020-10-15


## 8. Clean up stored procedure

Remove dbo.usp_AddDayToADate

In [11]:
%sql DROP PROCEDURE dbo.usp_AddDayToADate

Done.


[]

# Other useful info

* Check out %config SqlMagic and see what configuration is supported. 
* For more information about SqlMagic check out: https://pypi.org/project/ipython-sql/



In [12]:
%config SqlMagic

SqlMagic options
--------------
SqlMagic.autocommit=<Bool>
    Current: True
    Set autocommit mode
SqlMagic.autolimit=<Int>
    Current: 0
    Automatically limit the size of the returned result sets
SqlMagic.autopandas=<Bool>
    Current: False
    Return Pandas DataFrames instead of regular result sets
SqlMagic.column_local_vars=<Bool>
    Current: False
    Return data into local variables from column names
SqlMagic.displaycon=<Bool>
    Current: False
    Show connection string after execute
SqlMagic.displaylimit=<Int>
    Current: None
    Automatically limit the number of rows displayed (full result set is still
    stored)
SqlMagic.dsn_filename=<Unicode>
    Current: 'odbc.ini'
    Path to DSN file. When the first argument is of the form [section], a
    sqlalchemy connection string is formed from the matching section in the DSN
    file.
SqlMagic.feedback=<Bool>
    Current: True
    Print number of rows affected by DML
SqlMagic.short_errors=<Bool>
    Current: True
    Don't